In [3]:
import tensorflow as tf
import dask.dataframe as dd
from waymo_open_dataset import v2
import glob
import numpy as np
from waymo_open_dataset.utils.range_image_utils import build_camera_depth_image
from waymo_open_dataset.v2.perception.utils import lidar_utils

2024-11-29 11:27:10.240902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 11:27:11.414673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
search_path = '/home/meowater/Documents/ssd_drive/training/lidar/'
file_list = glob.glob(search_path + '*.parquet')


In [5]:
dataset_dir = '/home/meowater/Documents/ssd_drive/training/'

def read(tag: str, context_name) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/*{context_name}.parquet')
  return dd.read_parquet(paths)

In [28]:
def get_extrinsic(calibration):
  """Projects from vehicle coordinate system to image with global shutter.

  Arguments:
    calibration: Camera calibration details (including intrinsics/extrinsics).
  Returns:
    Array of shape [N, 3], with the latter dimension composed of (u, v, ok).
  """

  # Populate camera image metadata. Velocity and latency stats are filled with
  # zeroes.
  extrinsic = tf.reshape(
      tf.constant(list(calibration.extrinsic.transform), dtype=tf.float32),
      [1, 4, 4])

  # Perform projection and return projected image coordinates (u, v, ok).
  return extrinsic

In [9]:
# Lazily read DataFrames for all components.
# context_name = '15832924468527961_1564_160_1584_160'
context_name = '3270384983482134275_3220_000_3240_000'
# association_df = read('camera_to_lidar_box_association', context_name)
# cam_box_df = read('camera_box', context_name)
# cam_img_df = read('camera_image', context_name)
# lidar_box_df = read('lidar_box', context_name)
# cam_lidar_df = read('lidar_camera_projection', context_name)
#
# lidar_df = read('lidar', context_name)

# lidar_pose = read('lidar_pose', context_name)
# print(len(lidar_pose))


lidar_df = read('lidar', context_name)
lidar_calib_df = read('lidar_calibration', context_name)
camera_calib_df = read('camera_calibration', context_name)
vehicle_pose_df = read('vehicle_pose', context_name)
projections_df = read('lidar_camera_projection', context_name)


# print(len(lidar_df))
# # print(len(lidar_calib_df))
# # print(len(camera_calib_df))
# print(len(vehicle_pose_df))
# print(len(projections_df))
#
#
# df = v2.merge(vehicle_pose_df, lidar_df)
# print(len(df))
#
# df = v2.merge(df, projections_df)
# print(len(df))

In [10]:

_, r_lidar = next(iter(lidar_df.iterrows()))
lidar = v2.LiDARComponent.from_dict(r_lidar)

_, r_project = next(iter(projections_df.iterrows()))
projections = v2.LiDARCameraProjectionComponent.from_dict(r_project)



# lidar = v2.LiDARComponent.from_dict(r)
# lidar_calib = v2.LiDARCalibrationComponent.from_dict(r)
# camera_calib = v2.CameraCalibrationComponent.from_dict(r)
# # lidar_pose = v2.LiDARPoseComponent.from_dict(r)
# vehicle_pose = v2.VehiclePoseComponent.from_dict(r)
# camera_project = v2.LiDARCameraProjectionComponent.from_dict(r)



In [12]:
_, r_cam_config = next(iter(camera_calib_df.iterrows()))
cam_config = v2.CameraCalibrationComponent.from_dict(r_cam_config)

_, r_lidar_config = next(iter(lidar_calib_df.iterrows()))
lidar_config = v2.LiDARCalibrationComponent.from_dict(r_lidar_config)

In [34]:
print(projections)

LiDARCameraProjectionComponent(key=LaserKey(segment_context_name='3270384983482134275_3220_000_3240_000', frame_timestamp_micros=1549475204870940, laser_name=1), range_image_return1=CameraProjectionRangeImage(values=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), shape=array([  64, 2650,    6], dtype=int32)), range_image_return2=CameraProjectionRangeImage(values=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), shape=array([  64, 2650,    6], dtype=int32)))


In [40]:

lidar_coords = lidar_utils.convert_range_image_to_cartesian(lidar.range_image_return1, lidar_config)
lidar_coords = tf.expand_dims(lidar_coords, axis=0)
camera_extrinsic = get_extrinsic(cam_config)

camera_image_size = [cam_config.width, cam_config.height]
test2 = projections.range_image_return1.tensor
test2 = tf.expand_dims(test2, axis=0)
test = build_camera_depth_image(lidar_coords, camera_extrinsic,
                                camera_projection=test2,
                                camera_image_size=camera_image_size,
                                camera_name=1)

InvalidArgumentError: in user code:

    File "/home/meowater/PycharmProjects/DL_Final_Project/.venv/lib/python3.10/site-packages/waymo_open_dataset/utils/range_image_utils.py", line 462, in fn  *
        return scatter_nd_with_pool(index, value, camera_image_size, pool_method)
    File "/home/meowater/PycharmProjects/DL_Final_Project/.venv/lib/python3.10/site-packages/waymo_open_dataset/utils/range_image_utils.py", line 252, in scatter_nd_with_pool  *
        image = tf.scatter_nd(index_unique, value_pooled, shape)

    InvalidArgumentError: Value for attr 'Tindices' of float is not in the list of allowed values: int16, int32, int64
    	; NodeDef: {{node ScatterNd}}; Op<name=ScatterNd; signature=indices:Tindices, updates:T, shape:Tindices -> output:T; attr=T:type; attr=Tindices:type,allowed=[DT_INT16, DT_INT32, DT_INT64]> [Op:ScatterNd]
